# Image analysis of brightfields images 

Useful morphological features:
* **area**
* **perimeter**
* **convex area**: area of the convex hull that encloses the object.
* **convex perimeter** 
* **major axis length**: a measure of object length
* **minor axis length**: measure of object width
* **compactness**: defined as the ratio of the area of an object to the area of a circle with the same perimeter. Objects which have an elliptical shape, or a boundary that is irregular rather than smooth, will decrease the measure. Alternatively, can be computed as (perimeter)\*\*2/4\*pi\*area (to implement)
* **eccentricity**: the ratio of minor axis length/major axis length. When eccentricity == 0 -> circle
* **circularity**: computed as 4\*pi\*area/(convex perimeter)\*\*2. It's a measure of roundess or circularity where if == 1 -> circular object, but it's insensitive to irregular boundaries.
* **solidity**: area / convex area. A value of 1 signifies a solid object, and a value less than 1 will signify an object having an irregular boundary, or containing holes.


# Environment set up

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json
import warnings
from scipy import stats
import tifffile as tif

from skimage.measure import label, regionprops_table, regionprops
from skimage import img_as_float, util
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_holes, closing
from skimage.segmentation import clear_border

warnings.filterwarnings('ignore') 



import os

In [ ]:
def extra_prop(mask):
    
    from numpy import pi
    from skimage.measure import regionprops
    
    circ = lambda r: (4 * pi * r.area) / (r.perimeter * r.perimeter)
    
    regions = regionprops(mask.astype(np.uint8))
    
    compactness = [(i.perimeter**2)/(4 * pi * i.area) for i in regions if i.label != '0']
    convex_perimeter = [pi * i.equivalent_diameter_area for i in regions if i.label != '0']
    circularity = [circ(i) for i in regions if i.label != '0']
    solidity = [i.area/i.area_convex for i in regions if i.label != '0']
    
    return dict({'compactness': compactness, 'convex_perimeter': convex_perimeter, 'circularity': circularity, 'solidity': solidity})

In [ ]:
def segment_organoid(image, thr_mod = None, multiple_organoids = False):

    import numpy as np
    from skimage import img_as_float
    from skimage.filters import threshold_otsu
    from skimage.morphology import remove_small_holes, closing, square
    from skimage.segmentation import clear_border
    from skimage.measure import label
    
    if thr_mod == None:
        thr_mod = 0
    
    ch_0_float = img_as_float(image)

    try:
        threshold = threshold_otsu(ch_0_float) + thr_mod

        print(f"Threshold: {threshold}")
        bw = closing(ch_0_float < threshold, square(3))
        labels = label(clear_border(bw))

        labels_areas = np.unique(labels, return_counts=True)[1]
        max_area_label = np.max(labels_areas[1:])

        if multiple_organoids:

            labels_polished = label(remove_small_holes(labels, 2000))

            # remove labels smaller than 10000 (off target pixels)
            labels_areas = np.unique(labels, return_counts=True)[1]
            area_label_remove = labels_areas < 10000
            labels_remove = np.unique(labels_polished, return_counts=True)[0][area_label_remove]

            mask = np.in1d( labels_polished, labels_remove ).reshape( labels.shape )
            labels_polished[mask] = 0

        else:

            n_labels = np.unique(labels, return_counts=True)[0]
            max_label = np.unique(labels, return_counts=True)[0][ labels_areas == max_area_label ][0]

            labels_polished = np.where(labels != max_label, 0, max_label)
            labels_polished = remove_small_holes(labels_polished, 2000)

        return labels_polished

    except:
        
        print("There was an error in the segmentation, it will continue with the next image")

In [ ]:
def check_mask(image, mask, pixel_size = None, title = None):

    import matplotlib.pyplot as plt
    import copy
    from matplotlib_scalebar.scalebar import ScaleBar

    fig, ax = plt.subplots(1, 2, figsize = (20, 10))
    
    ax[0].imshow(image, cmap = 'Greys_r')

    if title != None:
        ax[0].set_title(title)
        
    ax[1].imshow(mask)

    if pixel_size != None:
        scalebar_0 = ScaleBar(pixel_size, units='um', color = 'black', box_alpha=0)
        scalebar_1 = ScaleBar(pixel_size, units='um', color = 'white', box_alpha=0)
    
        ax[0].add_artist(scalebar_0)
        ax[1].add_artist(scalebar_1)
    
    return plt.show()

def check_mask_countour(image, mask, pixel_size = None, title = None):

    import matplotlib.pyplot as plt
    import copy
    from matplotlib_scalebar.scalebar import ScaleBar

    fig, ax = plt.subplots(1, 1, figsize = (10, 10))
    
    ax.imshow(image, cmap = 'Greys_r')

    if title != None:
        ax.set_title(title)
    
    ax.contour(mask, alpha = 0.5, colors='red')
    
    if pixel_size != None:
        scalebar_0 = ScaleBar(pixel_size, units='um', color = 'white', box_alpha=0)
        ax.add_artist(scalebar_0)
    
    return plt.show()

In [ ]:
def get_image_data_tif(path):

    import tifffile as tif

    imgs_and_tags = tif.TiffFile(path)
    ch_0 = imgs_and_tags.pages[0].asarray()

    return ch_0

def get_image_data_czi(path, return_pixel_size = True):

    from aicsimageio import AICSImage
    from numpy import squeeze
    
    imgs = AICSImage(path)
    
    if return_pixel_size:
        pixel_size = imgs.physical_pixel_sizes.X
        
        return squeeze(imgs.get_image_data()), pixel_size
    
    else:
        return squeeze(imgs.get_image_data())

def get_mask_dict_from_folder(folder, thr_mod, threshdold_mod_dict = None, plot = True, multiple_organoids = False, only_dict_keys = False):

    img_dict_d0 = {}
    
    for i, img in enumerate(os.listdir(folder)):
        
        if img.endswith(".czi"):# and img in threshdold_mod_dict:
            
            img_dict_d0[img] = {}

            if threshdold_mod_dict != None:
                if img not in threshdold_mod_dict.keys():
                    img_dict_d0[img]['thresh_mod'] = thr_mod
                else:
                    img_dict_d0[img]['thresh_mod'] = threshdold_mod_dict[img]

            else:
                img_dict_d0[img]['thresh_mod'] = thr_mod
                
            print("File name: ", img)
            print("Threshold modifier: ", img_dict_d0[img]['thresh_mod'])

            img_dict_d0[img]["raw_image"], img_dict_d0[img]["pixel_size"]  = get_image_data_czi(f'{folder}/{img}', return_pixel_size = True)
            img_dict_d0[img]["mask"] = segment_organoid(img_dict_d0[img]["raw_image"], img_dict_d0[img]['thresh_mod'], multiple_organoids = multiple_organoids)

            if plot:
                try:
                    check_mask_countour(img_dict_d0[img]["raw_image"], img_dict_d0[img]["mask"], pixel_size =  img_dict_d0[img]["pixel_size"], title = img)
                except:
                    print("There was a problem in plotting")
            
    return img_dict_d0

In [ ]:
def get_image_data_czi(path, return_pixel_size = True):

    from aicsimageio import AICSImage
    from numpy import squeeze
    
    imgs = AICSImage(path)
    
    if return_pixel_size:
        pixel_size = imgs.physical_pixel_sizes.X
        
        return squeeze(imgs.get_image_data()), pixel_size
    
    else:
        return squeeze(imgs.get_image_data())

# HT day 0

In [ ]:
folder = "../../data/25_11_Day0/"

In [ ]:
px_size = []

for img in os.listdir(folder):
    img, px_s = get_image_data_czi(folder + '/' + img)
    px_size.append(px_s)

In [ ]:
np.unique(px_size)

In [ ]:
img_dict_d0 = {}

threshdold_mod_dict = {
    'MIX1_25_11_3.czi':0,
    'CTL04E_25_11_3.czi': 0,
    'MIX2_25_11_3.czi': 0,
    'MIX8_25_11_5.czi': 0,
    'MIX7_25_11_2.czi': 0,
    'KTD8.2_25_11_5.czi':0.05,
    'CTL04E_25_11_5.czi':0.05,
    'KTD8.2_25_11_4.czi':0.05,
    'MIX1_25_11_4.czi': 0.05,
    'MIX1_25_11_2.czi':0.05,
    'H1_25_11_5.czi':0.05,
    'MIX2_25_11_4.czi':-0.02,
    'MIX1_25_11_5.czi':0.05,

}

img_dict_d0[folder] = get_mask_dict_from_folder(folder, 0, threshdold_mod_dict)

# Day 2

In [ ]:
folder = "../../data/27_11_Day2/"

In [ ]:
px_size = []

for img in os.listdir(folder):
    img, px_s = get_image_data_czi(folder + '/' + img)
    px_size.append(px_s)

In [ ]:
np.unique(px_size)

In [ ]:
img_dict_d2 = {
}

threshdold_mod_dict = {     "MIX6_27_11_3.czi":-0.017,
    "KTD8.2_27_11_2.czi":0.01,
    "CHD8WT_27_11_1.czi":0.01,
    "MIX6_27_11_1.czi":0.01,
    "H1_27_11_5.czi":0.01
                      }

img_dict_d2[folder] = get_mask_dict_from_folder(folder, +0, threshdold_mod_dict)

# Day 4

In [ ]:
folder = "../../data/29_11_Day4/"

In [ ]:
px_size = []

for img in os.listdir(folder):
    img, px_s = get_image_data_czi(folder + '/' + img)
    px_size.append(px_s)

In [ ]:
np.unique(px_size)

In [ ]:
threshdold_mod_dict = {
    "MIX3_29_11_2.czi":0.02,
    "MIX7_29_11_5.czi":0.01,
    "MIX7_29_11_1.czi":0.04,
    "MIX2_29_11_2.czi":0.01,
    "MIX2_29_11_5.czi":0.01,
    "MIX3_29_11_1.czi":0.03,
    "MIX2_29_11_4.czi":0.02,
    "KTD8.2_29_11_1.czi":-0.01,
    "MIX7_29_11_2.czi":0.02,
    "MIX6_29_11_5.czi":0.02,
    "MIX6_29_11_4.czi":0.01,
}

img_dict_d4 = {}
img_dict_d4[folder] = get_mask_dict_from_folder(folder, +0, threshdold_mod_dict)

# Day 6

In [ ]:
folder = "../../data/01.12_Day6/"

In [ ]:
px_size = []

for img in os.listdir(folder):
    img, px_s = get_image_data_czi(folder + '/' + img)
    px_size.append(px_s)

In [ ]:
np.unique(px_size)

In [ ]:
threshdold_mod_dict = {
    "MIX2_01_12_3.czi":0.03,
    "CTL09A_01_12_5.czi":0.02,
    "MIX7_01_12_2.czi":0.02,
    "MIX6_01_12_4.czi":0.04,
    "MIX7_01_12_3.czi":0.03,
    "MIX1_01_12_2.czi":0.02,
    "MIX6_01_12_5.czi":0.02,
    "MIX6_01_12_2.czi":0.02,
    "MIX3_01_12_3.czi":0.05,
    "MIX3_01_12_1.czi":0.05,
    "MIX2_01_12_5.czi":0.02,
    "H1_01_12_5.czi":0.02,
    "CHD2WT_01_12_2.czi":0.02,
    "CTL05A_01_12_2.czi":0.03,
    "CTL01A_01_12_3.czi":0.03,
    "H1_01_12_1.czi":0.03,
    "CTL05A_01_12_3.czi":0.03,
    "MIX7_01_12_4.czi":0.03,
    "MIX3_01_12_2.czi":0.04,
    "MIX3_01_12_4.czi":0.04,
    "MIX2_01_12_1.czi":0.03,
    "MIX2_01_12_2.czi":0.03,
    "MIX6_01_12_3.czi":0.025,
    "MIX3_01_12_5.czi":0.04,
    "MIX2_01_12_4.czi":0.02,
    "MIX7_01_12_5.czi":0.02,
}

In [ ]:
img_dict_d6 = {}
img_dict_d6[folder] = get_mask_dict_from_folder(folder, +0.01, threshdold_mod_dict)

# Day 8

In [ ]:
folder = "../../data/03.12_Day8/"

In [ ]:
px_size = []

for img in os.listdir(folder):
    img, px_s = get_image_data_czi(folder + '/' + img)
    px_size.append(px_s)

In [ ]:
np.unique(px_size)

In [ ]:
threshdold_mod_dict = {
    "CTL04E_03-12_5.czi":0,
    "MIX2_03-12_5.czi":0.03,
    "MIX2_03-12_2.czi":0.02,
    "H1_03-12_5.czi":0,
    "MIX2_03-12_3.czi":0.02,
    "MIX6_03-12_2.czi":0.07,
    "MIX3_03-12_1.czi":0.08,
    "MIX2_03-12_4.czi":-0.02,
    "H1_03-12_4.czi":-0.015,
    "MIX7_03-12_5.czi":0,
    "CTL09A_03-12_3.czi":0.04,
    "MIX2_03-12_1.czi":0.05,
    "MIX6_03-12_3.czi":0.015,
    "CTL09A_03-12_1.czi":0,

}

In [ ]:
img_dict_d8 = {}
img_dict_d8[folder] = get_mask_dict_from_folder(folder, +0.05, threshdold_mod_dict)

# Day 10

In [ ]:
folder = "../../data/05_12_Day10/"

In [ ]:
px_size = []

for img in os.listdir(folder):
    img, px_s = get_image_data_czi(folder + '/' + img)
    px_size.append(px_s)

In [ ]:
np.unique(px_size)

In [ ]:
threshdold_mod_dict = {
    "MIX3_05_12_2.czi":0.02,
    "MIX2_05_12_3.czi":0.02,
    "MIX2_05_12_2.czi":-0.015,
    "MIX8_05_12_5.czi":0.06,
    "CTL09A_05_12_3.czi":0.01,
    "MIX6_05_12_5.czi":0.07,
    "KTD8.2_05_12_3.czi":0.03,
    "CTL01A_05_12_3.czi":0.065,

}

img_dict_d10 = {}
img_dict_d10[folder] = get_mask_dict_from_folder(folder, +0.05, threshdold_mod_dict)

# Compute the measurements

In [ ]:
all_dicts = [img_dict_d0, img_dict_d2, img_dict_d4, img_dict_d6, img_dict_d8, img_dict_d10]
days = ["0", '2', '4', '6', '8', '10']

In [ ]:
properties = ['label', 'area', 'area_filled', 'area_convex', 'perimeter', 'axis_major_length', 'axis_minor_length', 'equivalent_diameter_area', 'eccentricity']
extra_properties = [extra_prop]

In [ ]:
from tqdm import tqdm

In [ ]:
df_to_append = pd.DataFrame()
regionprops_table_df = pd.DataFrame()

with tqdm(total = 6) as t:
    
    for img_dict, d in zip(all_dicts, days):
        
        key = list(img_dict.keys())[0]

        for name in img_dict[key]:

            #print(name)

            #i = imgs_name.index(name)

            #try:
            regionprops_table_dict = regionprops_table(img_dict[key][name]["mask"].astype(np.uint8), properties = properties, extra_properties=extra_properties)

            regionprops_table_df = pd.DataFrame.from_dict(regionprops_table_dict)
            
            regionprops_table_df['Folder'] = key
            regionprops_table_df['FileName'] = name
            regionprops_table_df['Day'] = d
            regionprops_table_df['PixelSize'] = img_dict[key][name]['pixel_size']


            df_to_append = pd.concat([df_to_append, regionprops_table_df])

                
        t.update(1)

In [ ]:
df_to_append

In [ ]:
df_to_append['Line'] = df_to_append['FileName'].apply(lambda x: x.split('_')[0])
df_to_append['Replicate'] = df_to_append['FileName'].apply(lambda x: x.strip('.czi').split('_')[-1])
df_to_append['Replicate'] = df_to_append['Replicate'].str.replace('56', '5')

In [ ]:
df_to_append["Area(microns)"] = df_to_append["area"] * df_to_append["PixelSize"]
df_to_append["Perimeter(microns)"] = df_to_append["perimeter"] * df_to_append["PixelSize"]
df_to_append["AreaConvex(microns)"] = df_to_append["area_convex"] * df_to_append["PixelSize"]
df_to_append["EquivalentDiameterArea(microns)"] = df_to_append["equivalent_diameter_area"] * df_to_append["PixelSize"]

In [ ]:
df_to_append.Line.value_counts()

In [ ]:
for key in df_to_append.iloc[0].extra_prop.keys():
    df_to_append[key] = [i[key][0] for i in df_to_append.extra_prop]

In [ ]:
df_to_append['LineRep'] = df_to_append['Line'] + '_' + df_to_append['Replicate']

In [ ]:
#grouped = df_to_append.groupby('LineRep')

norm_area = pd.Series()
df_to_append.index = df_to_append.FileName
for LR in df_to_append['LineRep'].unique():
    
    sub = df_to_append[df_to_append['LineRep'] == LR]    
    prova = sub['area'] / sub[sub.Day == '0']['area'].values
    norm_area = norm_area._append(prova)

In [ ]:
df_to_append['AreaNorm'] = norm_area

# Save

In [ ]:
df_to_append.to_csv('organoidMultiplexing_growthCurves_quant.csv')